In [30]:
%cd /content/drive/My Drive/Colab Notebooks/DS_Lab/Data/

/content/drive/My Drive/Colab Notebooks/DS_Lab/Data


#Tiền xử lí: thu thập dữ liệu - Loại bỏ stop words & stemming --> Luu kq vào file:



In [0]:
#lay danh sach cac thu muc va newsgroup:
from os import listdir
def get_list_dirs_and_newsgroup():
  path = '20news-bydate/'
  parent_dirs = [path + dir_name + '/' for dir_name in listdir(path)]
  train_dir, test_dir = (parent_dirs[0], parent_dirs[1]) if 'train' in parent_dirs[0] else (parent_dirs[1], parent_dirs[0])
  list_group_name = [group_name for group_name in listdir(train_dir)] #day la danh sach cac group chung cho ca tap train va tap test
  list_group_name.sort()
  
  return train_dir, test_dir, parent_dirs, list_group_name

#Lay ra stop_words: 
def get_stop_words():
  with open(path + 'stop_word.txt') as file:
    stop_words = file.read().splitlines()
  file.close()
  return stop_words

#Import REgular Expression:
import re
#Import stemming package:
from nltk.stem.porter import PorterStemmer

#ham lay du lieu tu tap train va tap test:
def collect_data(parent_dir, list_group_name):
  data = [] #chua du lieu sau khi thu thap - moi van ban la mot phan tu
  list_group_name = get_list_dirs_and_newsgroup()[3]
  stop_words = get_stop_words()
  stemmer = PorterStemmer()

  #Xu li voi tung group trong list:
  for group_id, group_name in enumerate(list_group_name):
    label = group_id

    #lay duong dan toi cac file (ten_file, duong_dan)
    path_to_group = parent_dir + group_name
    files = [(file_name, path_to_group + '/'  + file_name ) for file_name in listdir(path_to_group)]
    files.sort() #sap xep cac tuple teo file_name

    #Doc DL tu cac file trong group va xu li:
    for file_name, file_path in files:
      with open(file_path, encoding='utf8', errors='ignore') as f:
        text = f.read()#chuyen het ve chu thuong
        
        #Loai bo cac ki tu khong co trong [a-zA-Z0-9] va khong phai stop word
        words_in_file = [stemmer.stem(word) for word in re.split('\W+', text) if word not in stop_words]

        #ket hop cac tu lai thanh 1 xau (cac tu cach nhau boi 1 khoang trang cach)
        content = ' '.join(words_in_file)

        #day du lieu vao data:
        data.append(str(label) + '<fff>' + file_name + '<fff>' + content)
      f.close()

  return data

#Ghi ket qua da thu thap duoc vao file:

def write_processed_data_to_file():
  with open('20news-train-processed.txt', 'w') as f:
    f.write('\n'.join(train_data)) #chuyen mang cac van ban thanh file gom cac van ban, moi ban ban tren 1 dong
  f.close()

  with open('20news-test-processed.txt', 'w') as f:
    f.write('\n'.join(test_data)) 
  f.close()

  with open('20news-full-processed.txt', 'w') as f:
    f.write('\n'.join(full_data))
  f.close()

#Tao tu dien - Tinh toan if-idf

In [0]:
#Tao tu dien - Su dung file full-precessed vua xu li duoc:
def create_dictionary():
  data_path = '20news-full-processed.txt'

  with open(data_path) as file:
    lines = file.read().split('\n') #moi van ban la mot phan tu cua mang lines
  file.close()

  corpus_size = len(lines) #kich thuoc tap D - tong so van ban

  #dem so van ban ma moi tu trong tu dien xuat hien
  from collections import defaultdict
  doc_count = defaultdict(int) #{tu: xuat hien trong bao nhieu van ban}

  for line in lines:
    features = line.split('<fff>') #tach lay phan van ban theo dinh dang truoc do da luu
    text = features[-1]
    words = list(set(text.split())) #thu duoc danh sach cac tu co trong mot van ban nao do
    for word in words:
      doc_count[word] +=1

  return doc_count #tu - so van ban ma no xuat hien


In [0]:
#tinh idf cua tu:
import numpy as np
def compute_idf(freq, corpus_size):
  return np.log10(corpus_size*1./freq)

def compute_word_idfs(doc_count):
  words_idfs = [(word, compute_idf(doc_freq, corpus_size))\
                for word, doc_freq in zip(doc_count.keys(), doc_count.values())\
                if doc_freq > 10 and not word.isdigit()]
  words_idfs.sort(key = lambda w:w[0])

  vocab_size = len(words_idfs)
  print("So tu trong tu dien: ", vocab_size)

  #Ghi ket qua tinh toan idfs ra file:
  with open('words_idfs.txt', 'w') as f:
    f.write('\n'.join([word + '<fff>' + str(idf) for word, idf in words_idfs]))

  return words_idfs

In [0]:
##Tinh tf-idf:
def get_tf_idf(data_path):

  #doc tu file words_idfs da luu truoc do de lay thong tin ve cac tu - tan suat xuat hien trong cac van ban
  with open('words_idfs.txt') as f:
    words_idfs = [(line.split('<fff>')[0], float(line.split('<fff>')[1]))\
                 for line in f.read().split('\n')]
    word_IDs  = dict([(word, index) for index, (word, idfs) in enumerate(words_idfs)])
    idfs = dict(words_idfs) #tu dien chua cac tu va tan suat xuat hien cua chung

  #doc tu file full-processed de lay toan bo van ban ra xem xet:
  with open(data_path) as f:
    documents =[(line.split('<fff>')[0],\
                 line.split('<fff>')[1], \
                 line.split('<fff>')[2])\
                for line in  f.read().split('\n')]
              
  data_tf_idf = []

  #xu li tung van ban
  for document in documents:
    label, doc_id, text = document
    words = [word for word in text.split() if word in idfs] #chi xet cac tu co tan suat > 10, khong phai so
    word_set = list(set(words))
    max_term_freq = max([words.count(word) for word in word_set])
    words_tfidfs = [] #mang chua tf-idf cua cac tu 
    sum_squares = 0.0

    #xet tf_idf cua tung tu trong van ban
    for word in word_set:
      term_freq = words.count(word)
      tf_idf_value = term_freq*1./max_term_freq*idfs[word]
      words_tfidfs.append((word_IDs[word], tf_idf_value))
      sum_squares +=tf_idf_value**2 #de chuan hoa 

    words_tfidfs_normalized = [str(index) + ':'\
                               + str(tf_idf_value/np.sqrt(sum_squares))\
                               for index, tf_idf_value in words_tfidfs]

    sparse_rep = ' '.join(words_tfidfs_normalized)
    data_tf_idf.append((label, doc_id, sparse_rep))

  return data_tf_idf


#Ham main:

In [0]:
if __name__ == '__main__':
  #goi thuc hien ham, tai day train_data, test_data tuong ung la 2 mang chua cac phan tu la cac van ban cua tap train, test tuong ung - cung voi nhan
  train_dir, test_dir, parent_dirs, list_group_name = get_list_dirs_and_newsgroup()
  train_data = collect_data(train_dir, list_group_name)
  test_data = collect_data(test_dir, list_group_name)
  full_data = train_data + test_data #noi hai list tren lai thanh list full - gom toan bo cac van ban

  write_processed_data_to_file() #Ghi kq ra file

  #Tao tu dien va tinh toan TF-IDF
  dictionary = create_dictionary()
  compute_word_idfs() #tinh idfs cua cac tu
  data_tf_idf = get_tf_idf(data_path = '20news-full-processed.txt') #tinh tf_idf tren toan bo du lieu:


  #Ghi ket qua cuoi cung ra file:
  with open('data_tf_idf.txt', 'w') as f:
    f.write('\n'.join([d_tf_idf[0] + '<fff>' + d_tf_idf[1] + '<fff>' + d_tf_idf[2] for d_tf_idf in data_tf_idf]))


